In [8]:
import sqlite3
from datetime import datetime
import numpy as np

# Connect to the SQLite database (or create a new one if it doesn't exist)
conn = sqlite3.connect('text_database.db')
cursor = conn.cursor()

text_types = ('личное сообщение', 'статья', 'новость', 'запись в личном блоге', 'публичный комментарий')
# Create the 'texts' table
cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS texts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        text TEXT,
        source TEXT,
        text_type TEXT CHECK(text_type IN {text_types}),
        date TIMESTAMP,
        url TEXT,
        platform TEXT,
        full_text TEXT,
        title TEXT,
        source_references TEXT,
        source_references_present BOOLEAN,
        logical_errors_present BOOLEAN,
        hatespeech_present BOOLEAN,
        hatespeech_indicators TEXT)
''')

# Create the 'manipulation_methods' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS manipulation_methods (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        manipulation_method_name TEXT)
''')

# Create the 'logical_fallacies' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS logical_fallacies (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        logical_fallacy_name TEXT)
''')

# Create the 'text_2_manipulation_methods' junction table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS text_2_manipulation_methods (
        text_id INTEGER,
        manipulation_method_id INTEGER,
        FOREIGN KEY (text_id) REFERENCES texts (id),
        FOREIGN KEY (manipulation_method_id) REFERENCES manipulation_methods (id),
        PRIMARY KEY (text_id, manipulation_method_id)
    )
''')


# Create the 'text_2_logical_fallacies' junction table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS text_2_logical_fallacies (
        text_id INTEGER,
        logical_fallacy_id INTEGER,
        FOREIGN KEY (text_id) REFERENCES texts (id),
        FOREIGN KEY (logical_fallacy_id) REFERENCES manipulation_methods (id),
        PRIMARY KEY (text_id, logical_fallacy_id)
    )
''')


# Example data to insert into the tables
example_data = {
    'text': 'Example texts text',
    'source': 'Example Source',
    'text_type': 'запись в личном блоге',
    'date': datetime.now(),
    'url': 'http://example.com',
    'platform': 'Web',
    'full_text': 'Full texts text',
    'title': 'Example Title',
    'source_references': 'Example source_references',
    'source_references_present': True,
    'logical_errors_present': False,
    'hatespeech_present': False,
    'hatespeech_indicators': 'example here'
}

manipulation_methods = ['Демонизация врага', 'Уловка Галилея']

# Insert the example data into the 'texts' table
cursor.execute('''
    INSERT INTO texts (
        text, source, text_type, date, url, platform, full_text, title, source_references,
        source_references_present, logical_errors_present,
        hatespeech_present, hatespeech_indicators
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', (
    example_data['text'], example_data['source'], example_data['text_type'],
    example_data['date'], example_data['url'], example_data['platform'],
    example_data['full_text'], example_data['title'], example_data['source_references'],
    example_data['source_references_present'], example_data['logical_errors_present'],
    example_data['hatespeech_present'], example_data['hatespeech_indicators']
))

# Insert manipulation methods into the 'manipulation_methods' table
for method in manipulation_methods:
    cursor.execute('INSERT INTO manipulation_methods (manipulation_method_name) VALUES (?)', (method,))

# Get the ID of the inserted texts article
text_id = cursor.lastrowid

# Associate manipulation methods with the texts article in the 'texts_manipulation' table
for method in manipulation_methods:
    cursor.execute('''
        INSERT INTO text_2_manipulation_methods (text_id, manipulation_method_id)
        VALUES (?, (SELECT id FROM manipulation_methods WHERE manipulation_method_name = ?))
    ''', (text_id, method))

# Commit changes and close the connection
conn.commit()
conn.close()

## on a sample

In [1]:
import pickle

In [2]:
sample = pickle.load(open('data/db_texts.p', 'rb'))

In [3]:
len(sample)

3000

In [4]:
sample[0]

{'created_at': datetime.datetime(2023, 9, 30, 19, 53, 2, tzinfo=datetime.timezone.utc),
 'message_url': 'https://t.me/rian_ru/216890',
 'type': 'video',
 'channel': 'rian_ru',
 'edited': False,
 'url': nan,
 'author': None,
 'tg_preview_text': 'Школьным учителям в Дагестане рекомендовано отказаться от использования иностранных мессенджеров, в том числе WhatsApp, и перейти на отечественные аналоги, сообщили РИА Новости в Минобрнауки республики',
 'source_url': nan,
 'text': nan,
 'meta_keywords': nan,
 'authors': nan,
 'twitter': nan,
 'site_name': nan}

In [33]:
import sys
sys.path.append('../flask_app/')
import get_chatgpt_criteria
import importlib
importlib.reload(get_chatgpt_criteria)
from get_chatgpt_criteria import *

In [34]:
process_result = process_text(sample[0]['tg_preview_text'])

In [59]:
import re

def parse_chatgpt_field(process_result, field):
    parsed_manipulation_methods = []
    lines = process_result[field].split('\n')
    for line in lines:
        method = re.findall(r'(\d+\.(.*?)\.)', line)
        if method:
            method_name = method[0][1].strip()
            method_explanation = line.replace(method[0][0], '').strip()
            parsed_manipulation_methods.append((method_name, method_explanation))
    return parsed_manipulation_methods

In [38]:
telegram_record = sample[0]

In [44]:
telegram_record

{'created_at': datetime.datetime(2023, 9, 30, 19, 53, 2, tzinfo=datetime.timezone.utc),
 'message_url': 'https://t.me/rian_ru/216890',
 'type': 'video',
 'channel': 'rian_ru',
 'edited': False,
 'url': nan,
 'author': None,
 'tg_preview_text': 'Школьным учителям в Дагестане рекомендовано отказаться от использования иностранных мессенджеров, в том числе WhatsApp, и перейти на отечественные аналоги, сообщили РИА Новости в Минобрнауки республики',
 'source_url': nan,
 'text': nan,
 'meta_keywords': nan,
 'authors': nan,
 'twitter': nan,
 'site_name': nan}

In [56]:
text2chatgpt_output = {}

In [60]:
len(text2chatgpt_output)

6

In [ ]:
import time
from tqdm import tqdm

manipulation_methods = []
all_parsed_manipulation_methods = []
all_parsed_logical_fallacies = []
errors = []
db_records = []

for i, telegram_record in enumerate(tqdm(sample)):
    db_record = {}
    try:
        if telegram_record['tg_preview_text'] in text2chatgpt_output:
            process_result = text2chatgpt_output[telegram_record['tg_preview_text']]
        else:
            process_result = process_text(telegram_record['tg_preview_text'])
            text2chatgpt_output[telegram_record['tg_preview_text']] = process_result
    except Exception as e:
        errors.append((i, e, 'failed to process with chatgpt'))
        print((i, e, 'failed to process with chatgpt'))
        process_result = {}
    try:
        db_record['text'] = telegram_record['tg_preview_text']
        db_record['source'] = telegram_record['channel']
        db_record['date'] = telegram_record['created_at'].strftime('%Y-%m-%d %H:%M:%S')
        db_record['url'] = telegram_record['message_url']
        db_record['platform'] = 'telegram'
        db_record['full_text'] = telegram_record['text'] if not np.isnan(telegram_record['text']) else ''
        db_record['title'] = telegram_record.get('title', '')
    except Exception as e:
        errors.append((i, e, 'failed to process telegram record'))
        print((i, e, 'failed to process telegram record'))
        db_record['text'] = 'unprocessed'
        db_record['source'] = 'unprocessed'
        db_record['date'] = 'unprocessed'
        db_record['url'] = 'unprocessed'
        db_record['platform'] = 'telegram'
        db_record['full_text'] = 'unprocessed'
        db_record['title'] = 'unprocessed'
    if process_result:
        try:
            db_record['text_type'] = process_result['text_type'].lower()
            db_record['source_references'] = process_result['references']
            db_record['source_references_present'] = process_result['references_present']
            db_record['logical_fallacies_present'] = process_result['logical_fallacies_present']
            db_record['hatespeech_present'] = 0 if any(x in process_result['hatespeech'] for x in ['не найден', 'не обнаружен', 'отсутствует']) else 1
            db_record['hatespeech_indicators'] = process_result['hatespeech']
            parsed_manipulation_methods = parse_chatgpt_field(process_result, 'manipulation_methods')
            all_parsed_manipulation_methods.append(parsed_manipulation_methods)
            parsed_logical_fallacies = parse_chatgpt_field(process_result, 'logical_fallacies')
            all_parsed_logical_fallacies.append(parsed_logical_fallacies)
        except Exception as e:
            errors.append((i, e, 'failed to parse chatgpt output'))
            print((i, e, 'failed to parse chatgpt output'))
    else:
        db_record['text_type'] = 'undefined'
        db_record['source_references'] = 'uncalculated'
        db_record['source_references_present'] = 0
        db_record['logical_fallacies_present'] = 0
        db_record['hatespeech_present'] = 0
        db_record['hatespeech_indicators'] = 'uncalculated'
    db_records.append(db_record)
    time.sleep(1)

  0%|▍                                                                                          | 14/3000 [01:23<8:25:44, 10.16s/it]

In [ ]:
import sqlite3
from datetime import datetime

# Connect to the SQLite database (or create a new one if it doesn't exist)
conn = sqlite3.connect('sample_database.db')
cursor = conn.cursor()

text_types = ('личное сообщение', 'статья', 'новость', 'запись в личном блоге', 'публичный комментарий')
# Create the 'texts' table
cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS texts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        text TEXT,
        source TEXT,
        text_type TEXT CHECK(text_type IN {text_types}),
        date TIMESTAMP,
        url TEXT,
        platform TEXT,
        full_text TEXT,
        title TEXT,
        source_references TEXT,
        source_references_present BOOLEAN,
        logical_fallacies_present BOOLEAN,
        hatespeech_present BOOLEAN,
        hatespeech_indicators TEXT)
''')

# Create the 'manipulation_methods' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS manipulation_methods (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        manipulation_method_name TEXT)
''')

# Create the 'logical_fallacies' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS logical_fallacies (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        logical_fallacy_name TEXT)
''')

# Create the 'text_2_manipulation_methods' junction table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS text_2_manipulation_methods (
        text_id INTEGER,
        manipulation_method_id INTEGER,
        manipulation_method_explanation TEXT,
        FOREIGN KEY (text_id) REFERENCES texts (id),
        FOREIGN KEY (manipulation_method_id) REFERENCES manipulation_methods (id),
        PRIMARY KEY (text_id, manipulation_method_id)
    )
''')


# Create the 'text_2_logical_fallacies' junction table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS text_2_logical_fallacies (
        text_id INTEGER,
        logical_fallacy_id INTEGER,
        logical_fallacy_explanation TEXT,
        FOREIGN KEY (text_id) REFERENCES texts (id),
        FOREIGN KEY (logical_fallacy_id) REFERENCES manipulation_methods (id),
        PRIMARY KEY (text_id, logical_fallacy_id)
    )
''')

manipulation_methods = ['Демонизация врага', 'Уловка Галилея']

# Example data to insert into the tables
example_data = {
    'text': 'Example texts text',
    'source': 'Example Source',
    'text_type': 'запись в личном блоге',
    'date': datetime.now(),
    'url': 'http://example.com',
    'platform': 'Web',
    'full_text': 'Full texts text',
    'title': 'Example Title',
    'source_references': 'Example source_references',
    'source_references_present': True,
    'logical_fallacies_present': False,
    'hatespeech_present': False,
    'hatespeech_indicators': 'example here'
}


# Insert the example data into the 'texts' table
cursor.execute('''
    INSERT INTO texts (
        text, source, text_type, date, url, platform, full_text, title, source_references,
        source_references_present, logical_fallacies_present,
        hatespeech_present, hatespeech_indicators
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', (
    example_data['text'], example_data['source'], example_data['text_type'],
    example_data['date'], example_data['url'], example_data['platform'],
    example_data['full_text'], example_data['title'], example_data['source_references'],
    example_data['source_references_present'], example_data['logical_fallacies_present'],
    example_data['hatespeech_present'], example_data['hatespeech_indicators']
))

# Insert manipulation methods into the 'manipulation_methods' table
for method in manipulation_methods:
    cursor.execute('INSERT INTO manipulation_methods (manipulation_method_name) VALUES (?)', (method,))

# Get the ID of the inserted texts article
text_id = cursor.lastrowid

# Associate manipulation methods with the texts article in the 'texts_manipulation' table
for method in manipulation_methods:
    cursor.execute('''
        INSERT INTO text_2_manipulation_methods (text_id, manipulation_method_id)
        VALUES (?, (SELECT id FROM manipulation_methods WHERE manipulation_method_name = ?))
    ''', (text_id, method))

# Commit changes and close the connection
conn.commit()
conn.close()